# 权限配置

In [14]:
import sagemaker
import os
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket_name=sess.default_bucket()
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::064542430558:role/service-role/AmazonSageMaker-ExecutionRole-20200803T154438
sagemaker bucket: sagemaker-us-west-2-064542430558
sagemaker session region: us-west-2


# 数据准备

In [7]:
# dataset used
dataset_name = 'SPDB'
# s3 key prefix for the data
s3_prefix = 'datasets/SPDB'
WORK_DIRECTORY = './data/'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=s3_prefix)
data_location

's3://sagemaker-us-west-2-064542430558/datasets/SPDB'

# 超参数定义

In [11]:
from sagemaker.huggingface import HuggingFace

# hyperparameters which are passed to the training job
hyperparameters={'reference_column':'ref',
                 'hypothesis_column':'hyp',
                 'train_file':'/opt/ml/input/data/train/cleaned_train.csv',
                 'validation_file':'/opt/ml/input/data/test/cleaned_test.csv',
                 'output_dir':'/opt/ml/model',
                 'do_train':True,
                 'do_eval':True,
                 'max_source_length': 128,
                 'max_target_length': 128,
                 'model_name_or_path': 'Langboat/mengzi-t5-base',
                 'learning_rate': 3e-4,
                 'num_train_epochs': 1,
                 'per_device_train_batch_size': 2,#16
                 'gradient_accumulation_steps':2, 
                 'save_strategy':'epoch',
                 'evaluation_strategy':'epoch',
                 'save_total_limit':1,
                 }
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}
# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='run_paraphrase.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',#'ml.p3dn.24xlarge'
        instance_count=1,
        role=role,
        max_run=24*60*60,
        transformers_version='4.6',
        pytorch_version='1.7',
        py_version='py36',
        volume_size=128,
        hyperparameters = hyperparameters,
#         distribution=distribution
)

# 模型训练

In [12]:
huggingface_estimator.fit(
  {'train': data_location+'/cleaned_train.csv',
   'test': data_location+'/cleaned_test.csv'}
)

2022-12-06 04:40:32 Starting - Starting the training job...
2022-12-06 04:40:57 Starting - Preparing the instances for trainingProfilerReport-1670301632: InProgress
.........
2022-12-06 04:42:21 Downloading - Downloading input data
2022-12-06 04:42:21 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-06 04:45:45,657 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-06 04:45:45,689 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-06 04:45:45,697 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-12-06 04:45:46,072 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2022-12-06 04:45:57 Training - Training image download completed. Training in 

In [15]:
training_job_name=huggingface_estimator.latest_training_job.name
model_s3_path='s3://{}/{}/output/model.tar.gz'.format(bucket_name, training_job_name)
model_s3_path

's3://sagemaker-us-west-2-064542430558/huggingface-pytorch-training-2022-12-06-04-40-31-766/output/model.tar.gz'

# 模型加载&部署

In [16]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
#    env= {'HF_TASK':'text-generation'},
   model_data=model_s3_path,  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.6",                           # Transformers version used
   pytorch_version="1.7",                                # PyTorch version used
   py_version='py36',                                    # Python version used
    
)

In [17]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)

-------!

# endpoint调用

In [8]:
# from sagemaker.huggingface.model import HuggingFacePredictor
# predictor=HuggingFacePredictor(endpoint_name='huggingface-pytorch-inference-2022-04-19-06-41-55-309')

In [18]:
import time
s=time.time()
profix=['货物相关',
 '状态_货物相关',
 '物流相关',
 '状态_物流相关',
 '产品相关',
 '评价',
 '订单相关',
 '其他',
 '状态_其他']
for i in profix:
    out=predictor.predict({
        'inputs': [i+': '+"这是否意味着椅子不能更换为新椅子？"],
        "parameters": {"max_length": 256},
    })
    break
e=time.time()
print(e-s)


0.7471308708190918


In [19]:
out

[{'generated_text': 'None'}]

# 本地训练

In [ ]:
!python scripts/run_paraphrase.py \
    --model_name_or_path t5-small \
    --do_train \
    --do_eval \
    --train_file data/parasci_train.csv \
    --validation_file data/parasci_val.csv \
    --output_dir /tmp/tst-summarization \
    --overwrite_output_dir \
    --save_strategy 'epoch' \
    --reference_column 'ref' \
    --num_train_epochs 1 \
    --hypothesis_column 'hyp' \
    --max_source_length 128 \
    --output_dir models \
    --max_target_length 128 \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate

In [7]:
!pip install -r scripts/requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 KB 4.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 KB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 10.9 MB/s eta 0:00:00
